In [ ]:
!python -m tf2onnx.convert --tflite lightspeech_quant.tflite --output lightspeech_quant.onnx
!python -m tf2onnx.convert --tflite mbmelgan.tflite --output mbmelgan.onnx

In [2]:
import onnxruntime as ort

model_path = "./lightspeech_quant.onnx"
session = ort.InferenceSession(model_path)

In [3]:
import numpy as np

input_ids = np.array([[2, 20, 9, 22, 7, 15, 20, 34, 22, 7, 13, 32, 13, 20, 6, 3, 13, 31, 20, 31, 3, 32, 13, 8, 20, 23, 13, 8, 3, 20, 39]], dtype=np.int32)
speaker_ids = np.array([0], dtype=np.int32)
speed_ratios = np.array([1.0], dtype=np.float32)
f0_ratios = np.array([1.0], dtype=np.float32)
energy_ratios = np.array([1.0], dtype=np.float32)

In [4]:
# Run inference
inputs = {
    "input_ids": input_ids,
    "speaker_ids": speaker_ids,
    "speed_ratios": speed_ratios,
    "f0_ratios": f0_ratios,
    "energy_ratios": energy_ratios,
}

mel_output, durations, _ = session.run(None, inputs)
print(mel_output)

[[[ 0.5466896   0.38584375  0.18734197 ...  0.06759626  0.09102625
    0.11805711]
  [ 0.51814175  0.4220223   0.22926603 ...  0.13438748  0.15225151
    0.18260399]
  [ 0.46104604  0.41187796  0.2093364  ...  0.25872326  0.27218843
    0.29437155]
  ...
  [-3.5913134  -3.517941   -3.5480285  ... -3.4597697  -3.4266446
   -3.4110146 ]
  [-3.6307237  -3.5784483  -3.6077275  ... -3.4405584  -3.398725
   -3.37717   ]
  [-3.659541   -3.581321   -3.580257   ... -3.3261874  -3.2860596
   -3.2700708 ]]]


In [5]:
mel_output.shape

(1, 234, 80)

In [6]:
durations

array([[ 6,  4,  7,  5,  8,  3,  5,  8,  4,  9,  5,  8, 10,  4,  2,  7,
         6, 10,  4,  9,  6,  8,  4,  6,  4,  6,  3,  7, 10,  7, 49]],
      dtype=int32)

In [7]:
model_path = "./mbmelgan.onnx"
session = ort.InferenceSession(model_path)

In [8]:
# Run inference
outputs = session.run(None, {"mels": mel_output,})
audio_tflite = outputs[0][0, :, 0]
print(audio_tflite)

[-4.5401962e-03 -3.5835924e-03 -2.8000979e-03 ... -2.7870834e-07
 -3.3027669e-07  1.4598308e-07]


In [9]:
audio_tflite.shape

(119808,)

In [10]:
from IPython.display import Audio

Audio(data=audio_tflite, rate=44100)